In [209]:
import pandas as pd
import numpy as np
import itertools
import os
import warnings  
warnings.filterwarnings('ignore')

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio
pio.renderers.default = "notebook"


from sklearn.linear_model import LinearRegression  
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import Perceptron
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC, LinearSVC
from sklearn.model_selection import cross_val_score
from sklearn.metrics import precision_score, recall_score, accuracy_score, r2_score
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from sklearn.preprocessing import MinMaxScaler





In [129]:
stat_filename = '../data/coronavirus-disease-covid-19-statistics-and-research.csv'

measures_filename = '../data/acaps_covid19_government_measures.xlsx'

cols = ['iso_code','continent','location','date'
         ,'total_cases','new_cases','total_deaths',
         'new_deaths','total_cases_per_million',
         'new_cases_per_million','total_deaths_per_million',
         'new_deaths_per_million','new_tests',
         'total_tests','total_tests_per_thousand',
         'new_tests_per_thousand','new_tests_smoothed',
         'new_tests_smoothed_per_thousand',
         'tests_units','stringency_index',
         'population','population_density',
         'median_age','aged_65_older','aged_70_older',
         'gdp_per_capita','extreme_poverty',
         'cardiovasc_death_rate','diabetes_prevalence',
         'female_smokers','male_smokers',
         'handwashing_facilities',
         'hospital_beds_per_thousand','life_expectancy']

df_stats =  pd.read_csv('https://covid.ourworldindata.org/data/owid-covid-data.csv',delimiter=',')
df_stats = df_stats[cols] 
df_measures = pd.read_excel('https://www.acaps.org/sites/acaps/files/resources/files/acaps_covid19_government_measures_dataset_0.xlsx', header=0,sheet_name='Dataset')

df_confirmed = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
df_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
df_recoveries = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')

print("Shape for Statistics DataFrame:", df_stats.shape)
print("Shape for Measures DataFrame:", df_measures.shape)

print("Shape for Confirmed DataFrame:", confirmed_df.shape)
print("Shape for Deaths DataFrame:", deaths_df.shape)
print("Shape for Recovories DataFrame:", recoveries_df.shape)


Shape for Statistics DataFrame: (166098, 34)
Shape for Measures DataFrame: (23923, 18)
Shape for Confirmed DataFrame: (284, 777)
Shape for Deaths DataFrame: (284, 777)
Shape for Recovories DataFrame: (269, 777)


In [130]:
df_confirmed.sample(10)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,2/23/22,2/24/22,2/25/22,2/26/22,2/27/22,2/28/22,3/1/22,3/2/22,3/3/22,3/4/22
264,Channel Islands,United Kingdom,49.372300,-2.36440,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
232,NaN,Slovakia,48.669000,19.69900,0,0,0,0,0,0,...,2052260,2071372,2083963,2102420,2112272,2119358,2134795,2150666,2164751,2178234
89,Unknown,China,NaN,NaN,0,0,0,0,0,0,...,0,0,0,0,58,0,0,0,0,0
238,NaN,Spain,40.463667,-3.74922,0,0,0,0,0,0,...,10914105,10949997,10977524,10977524,10977524,10977524,11036085,11054888,11078028,11100428
166,NaN,Latvia,56.879600,24.60320,0,0,0,0,0,0,...,612920,624008,633960,644077,649504,653120,662644,673218,682118,689794
70,Henan,China,37.895700,114.90420,5,5,9,32,83,128,...,2664,2664,2664,2664,2664,2666,2667,2672,2677,2680
209,NaN,Panama,8.538000,-80.78210,0,0,0,0,0,0,...,752907,753694,754362,754905,755262,755498,755853,756085,756539,757061
28,NaN,Bolivia,-16.290200,-63.58870,0,0,0,0,0,0,...,891851,891851,892537,893048,893223,893395,893512,893775,894200,894200
33,NaN,Bulgaria,42.733900,25.48580,0,0,0,0,0,0,...,1080571,1083425,1086328,1087796,1088520,1091279,1093920,1096194,1097298,1099423
120,French Guiana,France,3.933900,-53.12580,0,0,0,0,0,0,...,77575,77607,77625,77625,77625,77652,77652,77652,77733,77765


In [190]:
df_confirmed.describe()

,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,...,2/23/22,2/24/22,2/25/22,2/26/22,2/27/22,2/28/22,3/1/22,3/2/22,3/3/22,3/4/22
count,282.000000,282.000000,284.000000,284.000000,284.000000,284.000000,284.000000,284.00000,284.000000,284.000000,...,2.840000e+02,2.840000e+02,2.840000e+02,2.840000e+02,2.840000e+02,2.840000e+02,2.840000e+02,2.840000e+02,2.840000e+02,2.840000e+02
mean,20.106368,21.958718,1.961268,2.313380,3.323944,5.059859,7.464789,10.31338,19.647887,21.721831,...,1.513551e+06,1.519732e+06,1.525337e+06,1.530016e+06,1.533766e+06,1.538708e+06,1.544120e+06,1.549935e+06,1.556581e+06,1.562567e+06
std,25.841453,75.893366,26.402603,26.499491,32.992032,45.919425,64.175242,86.46777,212.163754,213.470672,...,6.095036e+06,6.105564e+06,6.115582e+06,6.122978e+06,6.127430e+06,6.136422e+06,6.143999e+06,6.152129e+06,6.162227e+06,6.171151e+06
min,-71.949900,-178.116500,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,4.643279,-22.036550,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,...,8.011000e+03,8.012500e+03,8.015750e+03,8.020250e+03,8.195750e+03,8.240000e+03,8.241500e+03,8.241500e+03,8.243000e+03,8.248750e+03
50%,21.607878,20.921188,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,...,6.634650e+04,6.696500e+04,6.873300e+04,6.873700e+04,6.874200e+04,6.875850e+04,6.877900e+04,7.003650e+04,7.006000e+04,7.007600e+04
75%,40.950592,84.992575,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,...,7.446248e+05,7.453292e+05,7.459942e+05,7.465328e+05,7.470960e+05,7.476448e+05,7.503412e+05,7.589795e+05,7.638605e+05,7.649270e+05
max,71.706900,178.065000,444.000000,444.000000,549.000000,761.000000,1058.000000,1423.00000,3554.000000,3554.000000,...,7.873894e+07,7.880629e+07,7.888116e+07,7.892980e+07,7.894787e+07,7.904433e+07,7.909136e+07,7.914372e+07,7.919601e+07,7.925051e+07


In [131]:
df_deaths.sample(10)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,2/23/22,2/24/22,2/25/22,2/26/22,2/27/22,2/28/22,3/1/22,3/2/22,3/3/22,3/4/22
279,NaN,West Bank and Gaza,31.952200,35.233200,0,0,0,0,0,0,...,5439,5458,5458,5458,5458,5501,5513,5532,5542,5557
265,Falkland Islands (Malvinas),United Kingdom,-51.796300,-59.523600,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
226,NaN,Saudi Arabia,23.885942,45.079162,0,0,0,0,0,0,...,8990,8991,8993,8994,8996,8998,9001,9002,9004,9005
149,NaN,Indonesia,-0.789300,113.921300,0,0,0,0,0,0,...,147025,147342,147586,147844,148073,148335,148660,149036,149268,149596
9,Australian Capital Territory,Australia,-35.473500,149.012400,0,0,0,0,0,0,...,33,33,33,33,34,34,34,34,34,34
54,Saskatchewan,Canada,52.939900,-106.450900,0,0,0,0,0,0,...,1054,1054,1091,1091,1091,1091,1091,1091,1091,1135
276,NaN,Vanuatu,-15.376700,166.959200,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,1
84,Shanghai,China,31.202000,121.449100,0,0,0,0,1,1,...,7,7,7,7,7,7,7,7,7,7
45,New Brunswick,Canada,46.565300,-66.461900,0,0,0,0,0,0,...,300,300,301,301,301,304,306,306,308,309
202,NaN,Niger,17.607789,8.081666,0,0,0,0,0,0,...,306,306,306,306,307,307,307,307,307,307


In [132]:
df_recoveries.sample(10)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,2/23/22,2/24/22,2/25/22,2/26/22,2/27/22,2/28/22,3/1/22,3/2/22,3/3/22,3/4/22
125,NaN,Guinea,9.945600,-9.696600,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
152,NaN,Lebanon,33.854700,35.862300,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
27,NaN,Bhutan,27.514200,90.433600,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
115,Wallis and Futuna,France,-14.293800,-178.116500,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,NaN,Argentina,-38.416100,-63.616700,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
22,NaN,Barbados,13.193900,-59.543200,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
100,NaN,Estonia,58.595300,25.013600,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
187,NaN,Niger,17.607789,8.081666,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
77,Zhejiang,China,29.183200,120.093400,0,0,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
264,NaN,West Bank and Gaza,31.952200,35.233200,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [186]:
df_confirmed_group = df_confirmed.groupby(by='Country/Region',as_index=False).sum()
df_deaths_group = df_deaths.groupby(by='Country/Region',as_index=False).sum()
df_recoveries_group = df_recoveries.groupby(by='Country/Region',as_index=False).sum()


df_active_group = pd.concat([df_deaths_group.iloc[: , :3],pd.DataFrame(df_confirmed_group.iloc[: , 3:] - (df_recoveries_group.iloc[: ,3:] +  df_deaths_group.iloc[:, 3:]))],axis=1)

In [202]:
stats = pd.DataFrame(columns=['Date','Confirmed','Deaths','Recovered','Active'])
stats['Date'] = df_confirmed.columns[4:]

stats['Confirmed'] = stats['Date'].apply(lambda x: df_confirmed[x].sum())
stats['Deaths'] = stats['Date'].apply(lambda x: df_deaths[x].sum())
stats['Recovered'] = stats['Date'].apply(lambda x: df_recoveries[x].sum())
stats.reset_index(drop=False, inplace=True)
stats['Active'] = stats['index'].apply(lambda x: (stats['Confirmed'][x]-(stats['Recovered'][x]+stats['Deaths'][x])))
stats.drop('index', axis = 1, inplace=True)
stats.head()

,Date,Confirmed,Deaths,Recovered,Active
0,1/22/20,557,17,30,510
1,1/23/20,657,18,32,607
2,1/24/20,944,26,39,879
3,1/25/20,1437,42,42,1353
4,1/26/20,2120,56,56,2008


In [215]:
latest_stats_fig = go.Figure()
latest_stats_fig.add_trace(go.Treemap(labels = ['Confirmed','Active','Recovered','Deaths'],
                                     parents = ['','Confirmed','Confirmed','Confirmed'],
                                     values = [stats['Confirmed'].sum(), stats['Active'].sum(), stats['Recovered'].sum(), stats['Deaths'].sum()],
                                      branchvalues="total", marker_colors = ['#073b4c','#ef476f','#06d6a0','#118ab2'],
                                      textinfo = "label+value",
                                      outsidetextfont = {"size": 30, "color": "darkblue"},
                                      marker = {"line": {"width": 2}},
                                        pathbar = {"visible": True}
                                     ))
latest_stats_fig.update_layout(height=300)
latest_stats_fig.show()